% # Pandas 数据规整

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm

## 数据导入与导出

```python
pd.read_csv('examples/ex1.csv')
pd.read_table('examples/ex1.csv', sep=',')

data.to_csv('examples/out.csv')


xlsx = pd.ExcelFile('examples/ex1.xlsx')
pd.read_excel(xlsx, 'Sheet1')
```



## 数据规整

### 行筛选

#### 条件筛选

```
df.query(expr, inplace=False, **kwargs)
```

#### 行去重

```
df.drop_duplicates()
```

#### 预览头部

```
df.head(n)
```

#### 预览尾部

```
df.tail(n)
```

#### 按比例采样

```
df.sample(frac=0.5)
```

#### 按数量采样

```
df.sample(n=10)
```

#### 按值取top

```
df.nlargest(n, 'value')
df.nsmallest(n, 'value')
```


In [3]:
df = pd.DataFrame({'A': range(1, 6),
                   'B': range(10, 0, -2),
                   'C C': range(10, 5, -1)})
df
df.query('A > B')
# 等价于
df[df.A > df.B]
df.query('B == `C C`')  # 列名中有空格时，要用背引号包围
# 等价于
df[df.B == df['C C']]

,A,B,C C
0,1,10,10
1,2,8,9
2,3,6,8
3,4,4,7
4,5,2,6


,A,B,C C
4,5,2,6


,A,B,C C
4,5,2,6


,A,B,C C
0,1,10,10


,A,B,C C
0,1,10,10


### 行排序

用sort_values方法，在排序时，任何缺失值默认都会被放到末尾

In [4]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame
frame.sort_values(by=['b','a'],ascending = [True,False])

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


### 列筛选

#### 根据列标签筛选

```
df.loc[:,'x2':'x4']
```

#### 根据列位置筛选

```
df.iloc[:,[1,2,5]]
```

#### 根据列标签特征筛选

```
df.filter(regex='regex')
```

#### 丢弃列

```
df.drop(columns=['Length','Height'])
```

#### 列重命名

```
df.rename(columns = {'y':'year'})
```

### 增加新列

```
df.assign(Area=lambda df: df.Length*df.Height)
```


In [5]:
df = pd.DataFrame({'temp_c': [17.0, 25.0]},
                   index=['Portland', 'Berkeley'])
df.assign(temp_f=lambda x: x.temp_c * 9 / 5 + 32)
df.assign(temp_f=df['temp_c'] * 9 / 5 + 32)
df.assign(temp_f=lambda x: x['temp_c'] * 9 / 5 + 32,
          temp_k=lambda x: (x['temp_f'] +  459.67) * 5 / 9)  # 引用了刚创建的列
# df.assign(temp_f=df['temp_c'] * 9 / 5 + 32,
#           temp_k=(df['temp_f'] +  459.67) * 5 / 9)  # 无法引用刚创建的列

,temp_c,temp_f
Portland,17.0,62.6
Berkeley,25.0,77.0


,temp_c,temp_f
Portland,17.0,62.6
Berkeley,25.0,77.0


,temp_c,temp_f,temp_k
Portland,17.0,62.6,290.15
Berkeley,25.0,77.0,298.15


### 层次化索引

可以通过unstack方法将这段数据重新安排到一个DataFrame中，unstack的逆运算是stack

In [6]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data
data.unstack()
data.unstack().stack()

a  1    1.013749
   2    1.078909
   3    0.655510
b  1    0.428180
   3   -1.144310
c  1   -0.322301
   2   -0.688062
d  2    0.043528
   3   -0.946829
dtype: float64

,1,2,3
a,1.013749,1.078909,0.655510
b,0.428180,NaN,-1.144310
c,-0.322301,-0.688062,NaN
d,NaN,0.043528,-0.946829


a  1    1.013749
   2    1.078909
   3    0.655510
b  1    0.428180
   3   -1.144310
c  1   -0.322301
   2   -0.688062
d  2    0.043528
   3   -0.946829
dtype: float64

In [7]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                      index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                      columns=[['Ohio', 'Ohio', 'Colorado'],
                               ['Green', 'Red', 'Green']])
frame
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

有时，你需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序。swaplevel接受两个级别编号或名称，并返回一个互换了级别的新对象

In [8]:
frame.swaplevel('key1', 'key2')
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

根据级别进行汇总


In [9]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

人们经常想要将DataFrame的一个或多个列当做行索引来用，或者可能希望将行索引变成DataFrame的列

In [10]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                       'c': ['one', 'one', 'one', 'two', 'two',
                             'two', 'two'],
                       'd': [0, 1, 2, 0, 1, 2, 3]})
frame
frame2 = frame.set_index(['c', 'd'])
frame2

# 默认情况下，那些列会从DataFrame中移除，但也可以将其保留下来
frame.set_index(['c', 'd'], drop=False)
# reset_index的功能跟set_index刚好相反，层次化索引的级别会被转移到列里面
frame2.reset_index()

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### 数据合并（按值）

pandas.merge 可根据一个或多个键将不同DataFrame中的行连接起来。SQL或其他关系型数据库的用户对此应该会比较熟悉，因为它实现的就是数据库的join操作。

pandas.merge函数，都可以作为DataFrame对象的merge方法。

默认情况下，merge做的是“内连接”；结果中的键是交集。其他方式还有"left"、"right"以及"outer"。外连接求取的是键的并集，组合了左连接和右连接的效果。

如果没有指定，merge就会将重叠列的列名当做键。不过，最好明确指定一下。如果两个对象的列名不同，也可以分别进行指定。

多对多连接产生的是行的笛卡尔积。要根据多个键进行合并，传入一个由列名组成的列表即可。

如果有重叠的列非指定的列，则列名后会加`_x`、`_y`作为后缀，也可以自己指定后缀，用suffixes参数。

In [11]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1
df2

pd.merge(df1, df2)
pd.merge(df1, df2, on='key')

df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

pd.merge(df1, df2, how='outer')

left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

df3.merge(df4, left_on='lkey', right_on='rkey')

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


### 数据拼接（按索引）

concat只能作为函数，不能作为方法

- 用 concat 函数，默认为纵向拼接，各列对齐（不同列取并集），各行堆叠，未重叠部分填充NaN。
- axis=1 表示横向拼接，各行对齐（不同行取并集），各列堆叠，未重叠部分填充NaN。
- join='inner'表示不同列或行时取交集而非并集。
- 假设你想要在连接轴上创建一个层次化索引。使用keys参数即可达到这个目的。

In [12]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(9).reshape(3, 3), index=['a', 'c','d'],
                   columns=['one','three', 'four'])
df1
df2
pd.concat([df1, df2])
pd.concat([df1, df2],axis=1)
pd.concat([df1, df2],join='inner')
pd.concat([df1, df2],axis=1,join='inner')
pd.concat([df1, df2], keys=['level1', 'level2'])
# df1.concat(df2)  # concat只能作为函数，不能作为方法

df3 = pd.DataFrame(np.arange(6).reshape(3, 2),
                   columns=['one', 'two'])
df4 = pd.DataFrame(4 + np.arange(6).reshape(3, 2),
                   columns=['one', 'two'])
df3
df4
pd.concat([df3, df4])  # 保留原索引，保留重复数据
pd.concat([df3, df4],ignore_index=True)  # 产生新索引，保留重复数据，相当于union all


,one,two
a,0,1
b,2,3
c,4,5


,one,three,four
a,5,6,7
c,8,9,10
d,11,12,13


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


,four,one,three,two
a,NaN,0,NaN,1.0
b,NaN,2,NaN,3.0
c,NaN,4,NaN,5.0
a,7.0,5,6.0,NaN
c,10.0,8,9.0,NaN
d,13.0,11,12.0,NaN


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


...

four  one  three  two
level1 a   NaN    0    NaN  1.0
       b   NaN    2    NaN  3.0
       c   NaN    4    NaN  5.0
level2 a   7.0    5    6.0  NaN
       c  10.0    8    9.0  NaN
       d  13.0   11   12.0  NaN

,one,two
0,0,1
1,2,3
2,4,5


,one,two
0,4,5
1,6,7
2,8,9


,one,two
0,0,1
1,2,3
2,4,5
0,4,5
1,6,7
2,8,9


,one,two
0,0,1
1,2,3
2,4,5
3,4,5
4,6,7
5,8,9


### 宽转长

可以是函数，也可以是方法。

```
pd.melt(DataFrame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None) -> DataFrame
```


In [13]:
df = pd.DataFrame({'group': ['foo', 'bar', 'baz'],
                    'A': [1, 2, 3],
                    'B': [4, 5, 6],
                    'C': [7, 8, 9]})
df
melted = pd.melt(df, ['group'])
melted
df.melt(id_vars=['group'],value_vars=['A','B','C'], 
        var_name='key',value_name='value')

,group,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


,group,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


,group,key,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


### 长转宽



```
df.pivot(index=None, columns=None, values=None) -> 'DataFrame'
```

index未指定时，用已存在的index，values未指定时，剩下所有列作为value。

In [14]:
reshaped = melted.pivot(index='group',columns='variable', values='value')
reshaped
reshaped.reset_index()  # 将索引变成普通列



variable,A,B,C
group,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


variable,group,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


### 哑变量



In [15]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df
pd.get_dummies(df['key'])

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


### 数据汇总

GroupBy的size方法给出分组大小.
你可以向groupby传入as_index=False以分组列自动作为行索引。

In [16]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df
grouped = df['data1'].groupby(df['key1'])
grouped.mean()
df.groupby(['key1', 'key2']).size()

(df['data1'].
    groupby([df['key1'], df['key2']]).
    mean())  
# 多分组，类似于R 管道操作 %>% 的写法，由于点的左右都不能断行，要在外面加括号
# 下面两种未加括号的写法都会报错

# df['data1']
# .groupby([df['key1'], df['key2']])
# .mean()  # 多分组

# df['data1'].
# groupby([df['key1'], df['key2']]).
# mean()  # 多分组

(df.
    groupby([df['key1'], df['key2']]).
    mean()) 
# 实质是在每一列，每个分组内都采取mean函数，并且将groupby的列作为index
(df.
    groupby([df['key1'], df['key2']]).
    mean().
    reset_index()) 

(df.
    groupby([df['key1'], df['key2']], as_index=False).
    mean()) 

,key1,key2,data1,data2
0,a,one,0.111401,0.258711
1,a,two,0.823555,-1.151463
2,b,one,-1.370137,-1.866663
3,b,two,-1.612589,0.167751
4,a,one,0.373553,0.119745


key1
a    0.436170
b   -1.491363
Name: data1, dtype: float64

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

key1  key2
a     one     0.242477
      two     0.823555
b     one    -1.370137
      two    -1.612589
Name: data1, dtype: float64

data1     data2
key1 key2                    
a    one   0.242477  0.189228
     two   0.823555 -1.151463
b    one  -1.370137 -1.866663
     two  -1.612589  0.167751

,key1,key2,data1,data2
0,a,one,0.242477,0.189228
1,a,two,0.823555,-1.151463
2,b,one,-1.370137,-1.866663
3,b,two,-1.612589,0.167751


,key1,key2,data1,data2
0,a,one,0.242477,0.189228
1,a,two,0.823555,-1.151463
2,b,one,-1.370137,-1.866663
3,b,two,-1.612589,0.167751


实际上，分组键可以是任何长度适当的数组

In [17]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.823555
            2006   -1.370137
Ohio        2005   -0.750594
            2006    0.373553
Name: data1, dtype: float64

通常，分组信息就位于相同的要处理DataFrame中。这里，你还可以将列名（可以是字符串、数字或其他Python对象）用作分组键。这意味着不用再外面写原数据框名，加方括号。

In [18]:
df.groupby('key1').mean()
df.groupby(['key1', 'key2']).mean()

,data1,data2
key1,,
a,0.436170,-0.257669
b,-1.491363,-0.849456


data1     data2
key1 key2                    
a    one   0.242477  0.189228
     two   0.823555 -1.151463
b    one  -1.370137 -1.866663
     two  -1.612589  0.167751

你可能已经注意到了，第一个例子在执行df.groupby('key1').mean()时，结果中没有key2列。这是因为df['key2']不是数值数据（俗称“麻烦列”），所以被从结果中排除了。默认情况下，所有数值列都会被聚合，虽然有时可能会被过滤为一个子集，稍后就会碰到。

如果用一个（单个字符串）或一组（字符串数组）列名对其进行索引，就能实现选取部分列进行聚合的目的。

```python
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
```
是以下代码的语法糖
```python
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])
```

#### 通过函数进行分组

将函数跟数组、列表、字典、Series混合使用也不是问题，因为任何东西在内部都会被转换为数组

In [19]:
people = pd.DataFrame(np.random.randn(5, 5),
                       columns=['a', 'b', 'c', 'd', 'e'],
                       index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people
people.groupby(len).sum()

,a,b,c,d,e
Joe,-1.552415,-0.290091,-1.081610,-0.050691,-0.603922
Steve,-0.961986,-0.389569,-1.465869,0.498314,-0.862406
Wes,0.291856,-1.403182,0.658457,1.059860,-1.084610
Jim,0.477706,0.286055,0.757025,-1.457597,-0.670113
Travis,-1.683025,-0.450932,-0.717360,0.605677,0.391176


,a,b,c,d,e
3,-0.782852,-1.407218,0.333872,-0.448429,-2.358645
5,-0.961986,-0.389569,-1.465869,0.498314,-0.862406
6,-1.683025,-0.450932,-0.717360,0.605677,0.391176


#### 聚合函数

常用聚合函数如下表。如果要使用你自己的聚合函数，只需将其传入aggregate或agg方法即可。自定义聚合函数要比表10-1中那些经过优化的函数慢得多。这是因为在构造中间分组数据块时存在非常大的开销（函数调用、数据重排等）。

![](groupby聚合函数.png)

In [20]:
df
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)
grouped.describe()

def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)
grouped.agg(['mean','std',peak_to_peak])

,key1,key2,data1,data2
0,a,one,0.111401,0.258711
1,a,two,0.823555,-1.151463
2,b,one,-1.370137,-1.866663
3,b,two,-1.612589,0.167751
4,a,one,0.373553,0.119745


key1
a    0.733555
b   -1.394382
Name: data1, dtype: float64

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.436170  0.360183  0.111401  0.242477  0.373553  0.598554   
b      2.0 -1.491363  0.171440 -1.612589 -1.551976 -1.491363 -1.430750   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.823555   3.0 -0.257669  0.777160 -1.151463 -0.515859  0.119745   
b    -1.370137   2.0 -0.849456  1.438548 -1.866663 -1.358059 -0.849456   

                          
           75%       max  
key1                      
a     0.189228  0.258711  
b    -0.340852  0.167751

,data1,data2
key1,,
a,0.712154,1.410173
b,0.242452,2.034414


data1                            data2                       
          mean       std peak_to_peak      mean       std peak_to_peak
key1                                                                  
a     0.436170  0.360183     0.712154 -0.257669  0.777160     1.410173
b    -1.491363  0.171440     0.242452 -0.849456  1.438548     2.034414

#### 离散化和面元划分

跟“区间”的数学符号一样，圆括号表示开端，而方括号则表示闭端（包括）。哪边是闭端可以通过right=False进行修改

In [21]:
ages = [17,20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32,101]
bins = [18, 25, 35, 60, 100]
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins)
pd.cut(ages, bins,labels=group_names)
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[NaN, (18.0, 25.0], (18.0, 25.0], (18.0, 25.0], (25.0, 35.0], ..., (60.0, 100.0], (35.0, 60.0], (35.0, 60.0], (25.0, 35.0], NaN]
Length: 14
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

[NaN, Youth, Youth, Youth, YoungAdult, ..., Senior, MiddleAged, MiddleAged, YoungAdult, NaN]
Length: 14
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

[NaN, [18.0, 26.0), [18.0, 26.0), [18.0, 26.0), [26.0, 36.0), ..., [61.0, 100.0), [36.0, 61.0), [36.0, 61.0), [26.0, 36.0), NaN]
Length: 14
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

如果向cut传入的是面元的数量而不是确切的面元边界，则它会根据数据的最小值和最大值计算等长面元。

In [22]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)  # 选项precision=2，限定小数只有两位。

[(0.03, 0.26], (0.5, 0.73], (0.73, 0.96], (0.03, 0.26], (0.73, 0.96], ..., (0.5, 0.73], (0.26, 0.5], (0.73, 0.96], (0.5, 0.73], (0.26, 0.5]]
Length: 20
Categories (4, interval[float64]): [(0.03, 0.26] < (0.26, 0.5] < (0.5, 0.73] < (0.73, 0.96]]

qcut是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分。根据数据的分布情况，cut可能无法使各个面元中含有相同数量的数据点。而qcut由于使用的是样本分位数，因此可以得到大小基本相等的面元.与cut类似，你也可以传递自定义的分位数（0到1之间的数值，包含端点）.

In [23]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4)
cats
pd.value_counts(cats)
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-0.624, 0.0248], (-3.542, -0.624], (0.696, 3.264], (-0.624, 0.0248], (0.696, 3.264], ..., (0.0248, 0.696], (-3.542, -0.624], (0.0248, 0.696], (-3.542, -0.624], (-0.624, 0.0248]]
Length: 1000
Categories (4, interval[float64]): [(-3.542, -0.624] < (-0.624, 0.0248] < (0.0248, 0.696] < (0.696, 3.264]]

(0.696, 3.264]      250
(0.0248, 0.696]     250
(-0.624, 0.0248]    250
(-3.542, -0.624]    250
dtype: int64

[(-1.249, 0.0248], (-1.249, 0.0248], (0.0248, 1.219], (-1.249, 0.0248], (1.219, 3.264], ..., (0.0248, 1.219], (-3.542, -1.249], (0.0248, 1.219], (-1.249, 0.0248], (-1.249, 0.0248]]
Length: 1000
Categories (4, interval[float64]): [(-3.542, -1.249] < (-1.249, 0.0248] < (0.0248, 1.219] < (1.219, 3.264]]

In [24]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                     'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)

def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.89, -1.952]",-2.555001,2.234924,31.0,0.027784
"(-1.952, -0.0222]",-3.079210,3.043311,458.0,-0.058192
"(-0.0222, 1.908]",-2.820099,2.870927,474.0,0.005331
"(1.908, 3.838]",-1.605074,1.790081,37.0,0.235776


,min,max,count,mean
data1,,,,
0,-2.871201,2.400820,100.0,-0.025361
1,-2.527763,3.043311,100.0,-0.188764
2,-2.864822,2.929944,100.0,0.026465
3,-2.355349,2.975277,100.0,0.037309
4,-3.079210,2.612861,100.0,-0.189946
5,-2.062142,2.870927,100.0,0.071723
6,-2.513542,2.675103,100.0,-0.026985
7,-2.820099,2.628203,100.0,0.167172
8,-2.578464,2.233214,100.0,-0.072022


### 透视表：同时实现长转宽与行列聚合

DataFrame有一个pivot_table方法，此外还有一个顶级的pandas.pivot_table函数。除能为groupby提供便利之外，pivot_table还可以添加分项小计，也叫做margins。

In [25]:
melted
melted.pivot(index='group',columns='variable', values='value')
# 与上面功能相同
melted.pivot_table(index='group',columns='variable', values='value')
# 增加聚合项，默认为平均值
melted.pivot_table(index='group',columns='variable', values='value',margins=True)
# 增加聚合项，合计
melted.pivot_table(index='group',columns='variable', values='value',margins=True,aggfunc=sum)

,group,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


variable,A,B,C
group,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


variable,A,B,C
group,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


variable,A,B,C,All
group,,,,
bar,2,5,8,5
baz,3,6,9,6
foo,1,4,7,4
All,2,5,8,5


variable,A,B,C,All
group,,,,
bar,2,5,8,15
baz,3,6,9,18
foo,1,4,7,12
All,6,15,24,45
